In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
orders_path = '../orders_after_cleaning.csv'
df_orders = pd.read_csv(orders_path,
                        dtype=
                        {'Customer number': object,
                         'Gender': object,
                         'PoR': object,
                         'Order number': object,
                         'Product number': object,
                         'Sub category': 'category',
                         'Category': 'category',
                         'Count': np.int16,
                         'Price': np.float64,
                         'EDT': np.int16,
                         'ADT': np.int16,
                         'RoR': object,
                         'Rating': np.int8})

In [3]:
df_orders["Order date"] = pd.to_datetime(df_orders["Order date"], format="%Y-%m-%d")

In [4]:
def calculate_mean(customer_number, dates):
    days = np.empty(len(dates))
    for i, date in enumerate(dates):
        if i < len(dates) - 1:
            days[i] = ((dates[i + 1] - dates[i]).astype('timedelta64[D]').astype(int))
    
    return round(days.mean()+days.std())

In [5]:
df_orders_per_customer = df_orders[['Customer number', 'Order number', 'Order date']].copy()
df_orders_per_customer = df_orders_per_customer.sort_values(['Customer number', 'Order date'])

In [6]:
df_orders_per_customer = df_orders_per_customer.drop_duplicates('Order number')

In [ ]:
customer_dates = {}
customer_numbers = df_orders_per_customer['Customer number']

for customer_no in customer_numbers.unique():
    customer_dates[customer_no] = df_orders_per_customer[customer_numbers == customer_no]['Order date'].values

In [ ]:
churn = []

for key, value in customer_dates.items():
    if (df_orders_per_customer['Order date'].iloc[-1] - customer_dates[key][-1]).days > calculate_mean(key, value):
        churn.append(1)
    else:
        churn.append(0)

In [ ]:
df_orders['Churn'] = churn
df_orders.head(10)